In [203]:
from selenium import webdriver
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time

In [204]:
driver = webdriver.Chrome("D:/chromedriver_win32/chromedriver.exe")

In [205]:
dataframe = pd.DataFrame(columns=["position","company","location","overview","deadline","url","job description"])

In [206]:
df = pd.DataFrame(columns=["images"])

In [207]:
pages = 8

for page in range(1,pages):
    
    url = "https://xpress.jobs/jobs?page=" + str(page) + "&Sectors=30"

    driver.get(url) 
    time.sleep(10)
    
    all_jobs = driver.find_elements_by_class_name('job_listing')
        
    for job in all_jobs:
        
        result_html = job.get_attribute('innerHTML')
        soup = BeautifulSoup(result_html, 'html.parser')
        
        try:
            position = job.find_element_by_class_name('job_listing-title').text
        except:
            position = 'None'
            
        try:
            company = soup.find(class_="job_listing-company").text.replace("\n","").strip()
        except:
            company = 'None'
            
        try:
            location = driver.find_element_by_xpath('//*[@id="job_listing-3354"]/div[2]/div[1]/div[2]').text
        except:
            location = 'None'

        try:
            overview = job.find_element_by_class_name('job_listing-overview').text
        except:
            overview = 'None'
            
        try:
            deadline = job.find_element_by_class_name('date').text
        except:
            deadline = 'None'
            
    
        link = soup.find('a',{'class':'job_listing-clickbox'})
        if link.get('href'):
            url = link.get('href').replace('../','https://xpress.jobs/')
            job_response = requests.get(url)
            job_data = job_response.text
            job_soup = BeautifulSoup(job_data, 'html.parser')
            job_description = job_soup.find('div',{'class':'job-description'}).text.replace("\n","").strip()
            i = 0
            for img in job_soup.findAll('img',attrs={'class':'alignnone size-full'}):
                image_temp = img.get('src')
               
        dataframe = dataframe.append({'position':position,"company":company,'location':location,"overview":overview,"deadline":deadline,"url":url,"job description":job_description},ignore_index=True)
        
        df = df.append({'images':image_temp},ignore_index=True)

In [208]:
dataframe.to_csv("D:/chromedriver_win32/Output.csv",index=False)
df.to_csv("D:/chromedriver_win32/Images.csv",index=False)

In [209]:
dataframe.shape

(64, 7)

In [210]:
df.shape

(64, 1)